In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/epi_r.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Columns: 680 entries, title to turkey
dtypes: float64(679), object(1)
memory usage: 104.0+ MB


In [6]:
for x in df.columns:
    print(x,", ",end='')

title , rating , calories , protein , fat , sodium , #cakeweek , #wasteless , 22-minute meals , 3-ingredient recipes , 30 days of groceries , advance prep required , alabama , alaska , alcoholic , almond , amaretto , anchovy , anise , anniversary , anthony bourdain , aperitif , appetizer , apple , apple juice , apricot , arizona , artichoke , arugula , asian pear , asparagus , aspen , atlanta , australia , avocado , back to school , backyard bbq , bacon , bake , banana , barley , basil , bass , bastille day , bean , beef , beef rib , beef shank , beef tenderloin , beer , beet , bell pepper , berry , beverly hills , birthday , biscuit , bitters , blackberry , blender , blue cheese , blueberry , boil , bok choy , bon appétit , bon app��tit , boston , bourbon , braise , bran , brandy , bread , breadcrumbs , breakfast , brie , brine , brisket , broccoli , broccoli rabe , broil , brooklyn , brown rice , brownie , brunch , brussel sprout , buffalo , buffet , bulgaria , bulgur , burrito , but

In [9]:
nulls = df.isnull().sum()
nulls[nulls > 0]

calories    4117
protein     4162
fat         4183
sodium      4119
dtype: int64

In [10]:
df.shape[0]

20052

In [12]:
df.rating.value_counts()

4.375    8019
3.750    5169
5.000    2719
0.000    1836
3.125    1489
2.500     532
1.250     164
1.875     124
Name: rating, dtype: int64

Want to make it into a binary classifier and cut it down to the thirty most valuable features.
Question 1 -- where to split it for binary classification, at 3.75,4...?
Question 2 -- how to find the thirty most valuable features.  For this maybe want to use corr and see if any corr are greater than 0.5 (which is a weak corr but still).  Maybe also create a decision tree and use that to find features.

In [13]:
df.select_dtypes(include='object')

,title
0,"Lentil, Apple, and Turkey Wrap"
1,Boudin Blanc Terrine with Red Onion Confit
2,Potato and Fennel Soup Hodge
3,Mahi-Mahi in Tomato Olive Sauce
4,Spinach Noodle Casserole
...,...
20047,Parmesan Puffs
20048,Artichoke and Parmesan Risotto
20049,Turkey Cream Puff Pie
20050,Snapper on Angel Hair with Citrus Cream


In [19]:
df2 = df.drop('title',axis=1).copy()

In [39]:
from sklearn.impute import KNNImputer
imp = KNNImputer()
df3 = pd.DataFrame(imp.fit_transform((df2.fillna(np.nan)).values),columns=df2.columns)
df3.loc[3]

rating          5.0
calories      545.8
protein        29.4
fat            28.6
sodium        789.8
              ...  
cookbooks       0.0
leftovers       0.0
snack           0.0
snack week      0.0
turkey          0.0
Name: 3, Length: 679, dtype: float64

In [44]:
# Using a decision tree to find the most important features
X = df3.drop('rating',axis=1)
y = df3.rating
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(X,y)
features = zip(list(df3.columns),list(dt.feature_importances_))
lst = (sorted(features, key = lambda x:x[1],reverse=True))[1:31]
features = [l[0] for l in lst]
features

['fat',
 'dried fruit',
 'calories',
 'protein',
 'hot pepper',
 'gouda',
 'hanukkah',
 'bok choy',
 'vegetable',
 'cobbler/crumble',
 'sukkot',
 'kentucky derby',
 'quiche',
 'bacon',
 'columbus',
 'dairy',
 'dill',
 'parsnip',
 'tree nut',
 'texas',
 'denver',
 'westwood',
 'kiwi',
 'custard',
 'midori',
 'healdsburg',
 'legume',
 'ramekin',
 'blueberry',
 'kid-friendly']

In [45]:
X = df3[features]
y = df3.rating

In [46]:
y.value_counts()

4.375    8019
3.750    5169
5.000    2719
0.000    1836
3.125    1489
2.500     532
1.250     164
1.875     124
Name: rating, dtype: int64

In [50]:
y = y.apply(lambda x: int(x>3.75))

In [51]:
# also normalize x

In [55]:
X = (X - X.mean())/X.std()

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 675)

In [57]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train,y_train)
svc.score(X_test,y_test)

0.5422587883320867